In [17]:
import pandas as pd
import json

In [14]:
def get_subject_and_question_into_json(file):
    subject = file['<< Contents'][0]
    question = file['<< Contents'][2]
    json_file = {}
    json_file['subject'] = subject
    json_file['question'] = question
    return json_file

def fill_with_preceding(row):
    for i in range(1, len(row)):
        if pd.isna(row[i]):
            row[i] = row[i-1]
    return row

def fill_fields(file):
    for i in range(1, len(file)):
        if '%' in file['Field_Type'].iloc[i-1] and pd.isna(file['Field_Type'].iloc[i]):
            file['Field_Type'].iloc[i] = 'Significance test for answer '+file['Field_Type'].iloc[i-2]
        elif pd.isna(file['Field_Type'].iloc[i]):
            file['Field_Type'].iloc[i] = file['Field_Type'].iloc[i-1] + ' ' + 'in %'
        
    return file

def get_json(file):
    end_row = file[file['<< Contents'].str.contains('Columns tested', case=False, na=False)].index[0]
    new_df = file.iloc[4:end_row]
    to_exclude = ['Unnamed: 0','Unnamed: 2']
    file_curated = new_df[[col for col in new_df.columns if col not in to_exclude]]
    file_curated = file_curated.rename(columns={'<< Contents': 'Field description'})
    file_curated = file_curated.apply(fill_with_preceding, axis=1)
    file_curated['Field description'].iloc[0] = 'Field'
    file_curated['Field description'].iloc[1] = 'Type'
    file_curated.columns = pd.MultiIndex.from_arrays([file_curated.iloc[0], file_curated.iloc[1]])
    file_curated.columns = [f'{i}_{j}' for i,j in file_curated.columns]
    file_curated.reset_index(drop=True, inplace=True)
    file_curated.drop([0,1], inplace=True)
    file_curated = fill_fields(file_curated)
    return file_curated.to_json(orient='records')

def compile_json(file):    
    json_file = get_subject_and_question_into_json(file)
    print(type(json_file))
    json_file['data'] = json.loads(get_json(file))
    return json_file

In [24]:
file = pd.read_csv('../data/savanta_data/polling_data0.csv')
topic_data = get_subject_and_question_into_json(file)
end_row = file[file['<< Contents'].str.contains('Columns tested', case=False, na=False)].index[0]
new_df = file.iloc[4:end_row]
to_exclude = ['Unnamed: 0','Unnamed: 2']
answers = new_df['<< Contents'].dropna().tolist()


{'subject': 'Homeless and Police PR - W184',
 'question': 'Q1. Summary : To what extent would you say that homelessness is, or is not, a problem in each of the following areas?'}

In [45]:
file_curated = new_df[[col for col in new_df.columns if col not in to_exclude]]
file_curated = file_curated.rename(columns={'<< Contents': 'Field description'})
file_curated = file_curated.apply(fill_with_preceding, axis=1)
if not file_curated.iloc[0].isna().all():
    file_curated.columns = pd.MultiIndex.from_arrays([file_curated.iloc[0], file_curated.iloc[1]])
    file_curated.columns = [f'{i}_{j}' for i,j in file_curated.columns]
else:
    file_curated.columns = file_curated.iloc[1]
groups = [x for x in file_curated.columns.to_list()]
print(groups)

[nan, 'Is a big problem', 'Is somewhat a problem', "Isn't much of a problem", "Isn't a problem at all", "Don't know", 'Sum: Problem', 'Sum: Not a problem']


/var/folders/96/98pztq455vz_10h20xcst8yh0000gn/T/ipykernel_17464/710541495.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.isna(row[i]):
/var/folders/96/98pztq455vz_10h20xcst8yh0000gn/T/ipykernel_17464/710541495.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[i] = row[i-1]
/var/folders/96/98pztq455vz_10h20xcst8yh0000gn/T/ipykernel_17464/710541495.py:12: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[i] = 

True

In [18]:
input_file = pd.read_csv('../data/savanta_data/polling_data0.csv')
output = compile_json(input_file)
output

<class 'dict'>


/var/folders/96/98pztq455vz_10h20xcst8yh0000gn/T/ipykernel_17464/710541495.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.isna(row[i]):
/var/folders/96/98pztq455vz_10h20xcst8yh0000gn/T/ipykernel_17464/710541495.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[i] = row[i-1]
/var/folders/96/98pztq455vz_10h20xcst8yh0000gn/T/ipykernel_17464/710541495.py:12: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[i] = 

{'subject': 'Homeless and Police PR - W184',
 'question': 'Q1. Summary : To what extent would you say that homelessness is, or is not, a problem in each of the following areas?',
 'data': [{'Field_Type': 'Significance Level: 95%',
   'nan_Is a big problem': '*a',
   'nan_Is somewhat a problem': '*b',
   "nan_Isn't much of a problem": '*c',
   "nan_Isn't a problem at all": '*d',
   "nan_Don't know": '*e',
   'nan_Sum: Problem': '*f',
   'nan_Sum: Not a problem': '*g'},
  {'Field_Type': "In the UK's big cities",
   'nan_Is a big problem': '1217',
   'nan_Is somewhat a problem': '701',
   "nan_Isn't much of a problem": '180',
   "nan_Isn't a problem at all": '65',
   "nan_Don't know": '73',
   'nan_Sum: Problem': '1917',
   'nan_Sum: Not a problem': '245'},
  {'Field_Type': "In the UK's big cities in %",
   'nan_Is a big problem': '0.54',
   'nan_Is somewhat a problem': '0.31',
   "nan_Isn't much of a problem": '0.08',
   "nan_Isn't a problem at all": '0.03',
   "nan_Don't know": '0.03',
